# MicroPython玩 otto 機器人

## OTTO (O++O) 機器人

![otto](image/my-otto.jpg)

### 使用4個servo (伺服馬達)、一個超音波sensor

- 超音波sensor目前只是當眼睛裝飾用、未來在機器人走路時才會有避障的功能
- 超音波使用腳位為 (D7, D8) --> (trig, echo)

- 目前使用的的馬達(SG90)如下圖:

![SG90](image/SG90-servo.jpg)

- 4個servo分別是：左腿、左腳、右腿、右腳
- 對應的GPIO為 D1, D2, D3, D4

In [1]:
%websocketconnect --password 1234 ws://192.168.43.230:8266

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [2]:
# 定義
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM


In [3]:
# angle define
ang0=40
ang90=77
ang180=115

### 開始測試四個關節


In [4]:
# 定義四個關節名稱
Lleg = PWM(Pin(D1), freq=50) # up servo of left foot
Lpad = PWM(Pin(D2), freq=50) # bottom servo of left foot (Footpad)
Rleg = PWM(Pin(D3), freq=50) # up servo of right foot
Rpad = PWM(Pin(D4), freq=50) # bottom servo of right foot (Footpad)


In [5]:
# 立正
Lleg.duty(ang90)
Lpad.duty(ang90)
Rleg.duty(ang90)
Rpad.duty(ang90)

In [6]:
# L-leg
Lleg.duty(ang0)
time.sleep(2)
Lleg.duty(ang180)
time.sleep(2)
Lleg.duty(ang90)

In [7]:
# L-foot
Lpad.duty(ang0)
time.sleep(2)
Lpad.duty(ang180)
time.sleep(2)
Lpad.duty(ang90)

.

In [8]:
# R-pad
Rleg.duty(ang0)
time.sleep(2)
Rleg.duty(ang180)
time.sleep(2)
Rleg.duty(ang90)

.

In [9]:
# R-foot
Rpad.duty(ang0)
time.sleep(2)
Rpad.duty(ang180)
time.sleep(2)
Rpad.duty(ang90)

.

## 測試眼睛 - 超音波sensor

In [10]:
def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58

In [16]:
print(ping(trigPin=D7,echoPin=D8))

7.63793


----
### 定義函式來控制馬達角度

In [46]:
def servo_ang(pwn_pin, ang):
    ang0=40
    ang90=77
    ang180=115
    value = int(ang0+(ang180-ang0)*ang/180)
    pwn_pin.duty(value)